In [1]:
import numpy as np
import pandas as pd

# Simulation Idea

three types of personalities having different effects on teamwork

1. the "normal" guy just contributes his performance 1 point (~ on average)

2. the "overachiever" exhibits double the normal perfomance, but with his ellbow mentality he lowers the productivity of every other team member with a malus of -0.25 points

3. the "charismatic idiot" achieves only half of the normal performance, but he motivates the team leading to a bonus of 0.25 points per other team member

With a distribution of 50% "normal", 25% "overachiever" and 25% "charismatic idiot" random teams with a size of 5 persons are formed. The winning teams out of competition groups of 4 teams get one promotion each. To choose the promotion candidate from a winning team, each person's single performance is deciding, thus ignoring previously mentioned team effects (bonus/malus). However, the "charismatic idiot" gets a specific bonus of +2 points just for the promotion selection.

The open postions are then refilled by new persons from the same population distribution and a cycle of competion, promotion and substitution presents the simulation loop. (the structure containing all groups is called corporate in this example)

In [2]:
def clipped_normal_distriution(x0,sigma):
    """a strictly positive "normal distribution" as the normal distribution is resampled, 
    whenever a negative value appears
    """
    value = np.random.normal(x0,sigma)
    return(value if value>=0 else clipped_normal_distriution(x0,sigma))

class social_simulation:
    """meritocracy game
    """
    
    def __init__(self,corporate_size=1000,group_size=5,group_competition_number=4):
        """initialize the simulation, by setting global parameters like the corporate and group size
        (corporate size should be a multiple of group size)
        """
        self.corporate_size=corporate_size
        self.group_size=group_size
        self.number_of_groups=int(self.corporate_size/self.group_size)
        self.group_competition_number=group_competition_number
        self._check_parameter_consistency()

        # further default parameters
        self.random_performance_deviation=0.25

        self.overachiever_performance_factor=2
        self.charismatc_idiot_performance_factor=0.5
        
        self.overachiever_group_malus=-0.25
        self.charismatic_idiot_group_bonus=0.25

        self.charismatic_idiot_promotion_bonus=2

        #further technical parameters 
        self._index_id=0  # anything starting with underscores, means it should not be accessed by a typical user
        self.save_folder=None
        
    def _check_parameter_consistency(self):
        """assert that corporate_size, group_size and group_competition_number fulfil all conditions of multiples
        """
        if self.corporate_size%self.group_size != 0:
            print("corporate size "+str(self.corporate_size)+
                  " is not divisible into an integer number of groups of size "+str(self.group_size))
            adjusted_corp_size=int(np.ceil(self.corporate_size/self.group_size)*self.group_size) 
            print("adjusted corporate size: "+str(adjusted_corp_size))
            self.corporate_size=adjusted_corp_size
            self.number_of_groups=int(self.corporate_size/self.group_size)
            print()
            
        if self.number_of_groups%self.group_competition_number !=0:
            print("number of groups "+str(self.number_of_groups)+
                  " is not divisible into an integer number of group competitions each containing "+
                  str(self.group_competition_number)+ " groups")
            adjusted_num_groups=int(np.ceil(self.number_of_groups/self.group_competition_number)*self.group_competition_number)
            adjusted_corp_size=int(adjusted_num_groups*self.group_size)
            print("adjusted number of groups: "+str(adjusted_num_groups))
            print("leading to adjusted corporate size: "+str(adjusted_corp_size))
            self.corporate_size=adjusted_corp_size
            self.number_of_groups=adjusted_num_groups
            print()
                    
    @staticmethod
    def _get_personality_from_index(index):
        """helper function to get personality distribution from random integers with uniform probability
        0,1 (50%) -> normal , 2 (25%) -> overachiever , 3 (25%) -> charismatic idiot
        """
        if index <2:
            personality="normal"
        elif index==2:
            personality="overachiever"
        elif index==3:
            personality="charismatic idiot"
        return personality

    @staticmethod
    def _get_performance_scores(random_performance_deviation,number_of_trainees):
        """the standard performance is 1 (100%) and the random_performance_deviation represents the
        standard deviation of a strictly positive "normal distribution"
        """
        return [clipped_normal_distriution(1,random_performance_deviation) for i in range(number_of_trainees)]

    def create_trainees(self,number_of_trainees,random_performance_deviation):
        """the given number of trainees is created, by three lists containing:
        the trainee ID, the trainee personality and the trainee performance
        """
        # trainee index / ID
        trainee_id=[i for i in range(self._index_id,self._index_id+number_of_trainees)]
        self._index_id += number_of_trainees

        # create personalities
        personality_index=np.random.randint(0,4,number_of_trainees)
        personality=[self._get_personality_from_index(i) for i in personality_index]

        # create corresponding performance scores
        performance=self._get_performance_scores(random_performance_deviation,number_of_trainees)
        
        for i in range(len(personality)):
            if personality[i]=="normal":
                pass
            elif personality[i]=="overachiever":
                performance[i] *= self.overachiever_performance_factor
            elif personality[i]=="charismatic idiot":
                performance[i] *= self.charismatc_idiot_performance_factor

        return trainee_id,personality,performance
                
    def initialize_groups(self):
        """Create the goup structure as a list of lists, with each list having the length of the group size,
        containing the trainee IDs belonging to that group. Additionally a list for look-up is created,
        which links each trainee ID to the corresponding group index.
        """
        self.group_lookup = [i//self.group_size for i in range(self.corporate_size)]
        self.groups=[]
        for i in range(self.corporate_size):
            if i%self.group_size == 0:
                self.groups.append([])
            self.groups[-1].append(self.trainee_id[i])

    def initialize_corporation(self):
        """initialize corporation by creating the starting trainees and the group structure,
        as well as initializing further needed variables
        """
        trainee_id,personality,performance=self.create_trainees(self.corporate_size,self.random_performance_deviation)
        self.trainee_id=trainee_id
        self.personality=personality
        self.performance=performance
        self.initialize_groups()

        self.entry_time=[0 for i in range(self.corporate_size)]
        self.promoted=[None for i in range(self.corporate_size)]
        self.time_step=0

        self.group_score=np.empty(self.number_of_groups)
        self.group_share_normal=np.empty(self.number_of_groups)
        self.group_share_charismatic_idiot=np.empty(self.number_of_groups)
        self.group_share_overachiever=np.empty(self.number_of_groups)
    
    def calculate_group_performance(self,group_index):
        """calculate the group score of a single group by summing all individual performance scores
        of trainees in the group and also summing all personality effects
        """
        group=self.groups[group_index]
        personalities_in_group=np.array([self.personality[i] for i in group])    
        number_of_overachievers=sum(personalities_in_group=="overachiever")
        number_of_charismatic_idiots=sum(personalities_in_group=="charismatic idiot")
        performance_sum=sum([self.performance[i] for i in group]) 
        bonus_sum=(len(group)-1)*number_of_charismatic_idiots* self.charismatic_idiot_group_bonus
        malus_sum=(len(group)-1)*number_of_overachievers*self.overachiever_group_malus
        
        group_performance = performance_sum + bonus_sum + malus_sum
        
        # save the calculated properties
        self.group_share_charismatic_idiot[group_index]=number_of_charismatic_idiots/self.group_size
        self.group_share_overachiever[group_index]=number_of_overachievers/self.group_size
        self.group_share_normal[group_index]=(self.group_size-number_of_overachievers-number_of_charismatic_idiots)/self.group_size
        self.group_score[group_index]=group_performance
        
        return group_performance
            
    def select_promotion(self,group):
        """determine for a given group, who will be promoted and return the corresponding 
        local index in the group and global index (trainee_id)
        """
        promotion_score=np.empty(len(group))
        for counter,i in enumerate(group):
            if self.personality[i]=="charismatic idiot":
                promotion_score[counter]=self.performance[i] +self.charismatic_idiot_promotion_bonus
            else:
                promotion_score[counter]=self.performance[i]
        promotion_group_index=np.argmax(promotion_score)
        promotion_global_index=group[promotion_group_index]
        return promotion_group_index,promotion_global_index
            
    def group_competition_round(self):
        """iterate over all groups and let them compete (biggest group_score wins) with each other in 
        several "battle royales" until every group has competed. Here, "battle royales" means a competition
        of "group_competition_number" participants resulting in only one winner
        """
        #random order of groups, such that it is not always the same groups competing with each other
        random_for_sorting=np.random.uniform(size=self.number_of_groups)
        competition_order=np.argsort(random_for_sorting)

        #execute group competitions
        number_of_competitions=int(self.number_of_groups/self.group_competition_number)
        winning_groups=[]
        for i in range(number_of_competitions):
            competing_groups=[]
            group_scores=np.empty(self.group_competition_number)
            for j in range(self.group_competition_number):
                competing_groups.append(competition_order[i*self.group_competition_number+j])
                group_scores[j]=self.calculate_group_performance(competing_groups[-1])
                
            winning_index=np.argmax(group_scores)
            winning_groups.append(competing_groups[winning_index])

        #save data about which groups win
        self.group_winning=np.zeros(self.number_of_groups,dtype=bool)
        for i in winning_groups:
            self.group_winning[i]=True
        return winning_groups
        
    def promotion_and_substitution_round(self,winning_groups):
        """the promotion of a single trainee of each winning group
        and in turn the hiring of a new trainee filling the position  
        marks the beginning of a new time step, as now with different people 
        the corporation is in a different state/configuration
        """
        self.time_step+=1
        
        number_of_new_trainees=len(winning_groups)
        trainee_id,personality,performance=self.create_trainees(number_of_new_trainees,self.random_performance_deviation)
        self.trainee_id+=trainee_id
        self.personality+=personality
        self.performance+=performance
        self.entry_time+=[self.time_step for i in range(number_of_new_trainees)]
        self.promoted+=[None for i in range(number_of_new_trainees)]

        newgroup_lookup=[]
        for counter,i in enumerate(winning_groups):
            winner_local,winner_global=self.select_promotion(self.groups[i])
            self.promoted[winner_global]=self.time_step
            self.groups[i][winner_local]=trainee_id[counter]
            newgroup_lookup.append(i)
        self.group_lookup += newgroup_lookup

    def set_save_location(self,folderpath):
        """saving the data of the simulation contains static and the dynamic parts of information.
        Saving the static data creates only one file, while the saving the dynamic part creates one file for
        each time_step.
        """
        if not folderpath[-1]=="/":
            folderpath+="/"
        self.save_folder=folderpath

    def save_total(self):
        """saving the static data, which is focused on the properties of the trainees
        """
        if self.save_folder is None:
            raise Exception("save folder is not defined, please use set_save_loaction before")
        #trainee data
        save_dict=dict()
        save_dict["trainee_id"]=self.trainee_id
        save_dict["performance"]=self.performance
        save_dict["personality"]=self.personality
        save_dict["entry_time"]=self.entry_time
        save_dict["promotion_time"]=self.promoted
        save_dict["group_lookup"]=self.group_lookup
        table=pd.DataFrame.from_dict(save_dict)
        table.to_csv(self.save_folder+"trainee_data.csv",index=False)
    
    def save_moment(self):
        """saving the dynamic part of the simulation, which represents the properties of groups,
        as they can change with each time step
        """
        if self.save_folder is None:
            raise Exception("save folder is not defined, please use set_save_loaction before")
        #group data 
        save_dict=dict()
        save_dict["group_id"]=np.arange(self.number_of_groups)
        save_dict["group_score"]=self.group_score
        save_dict["group_share_normal"]=self.group_share_normal
        save_dict["group_share_overachiever"]=self.group_share_overachiever
        save_dict["group_share_charismatic_idiot"]=self.group_share_charismatic_idiot
        save_dict["group_winning"]=self.group_winning
        save_trainee_IDs=[]
        for i in self.groups:
            save_trainee_IDs.append(str(i))
        save_dict["group_trainee_ids"]=save_trainee_IDs

        table=pd.DataFrame.from_dict(save_dict)
        table.to_csv(self.save_folder+"group_data_time_"+str(self.time_step).zfill(3)+".csv",index=False)     

In [3]:
sim=social_simulation(corporate_size=1000,group_size=5)

sim.set_save_location("meritocracy_simulation")
# (one needs to create the folder for the save_location)

number_of_timesteps=30
sim.initialize_corporation()
for i in range(number_of_timesteps):
    winners=sim.group_competition_round()
    sim.save_moment()
    sim.promotion_and_substitution_round(winners)

sim.save_total()